# Dynamic Programming (Value Iteration)

### Return
It's defined as the total discounted reward received from the environment on an infinite horizon
$$\text{Return}=\sum_{i=0}^{\infty}\gamma^i r_i \therefore [r_0 + \gamma.r_1 + \gamma^2.r_2 + \gamma^3.r_3 + ...\gamma^\infty.r_\infty]$$
The reasons for discounted reward are:
* Avoid inifintie reward
* Model the behaviour of the agent that will prefer imediate rewards ($\gamma=0$) or long term rewards ($\gamma=1$)

### Value Function
It's the return expected value.
$$V^\pi(s)=\mathbb{E}\left[\sum_{i=0}^{\infty} \gamma^i r_i\right] s \in S$$

### Value Iteration
This algorithm use the Bellman expectation equation to iteratively find the optimum Value function. The algorithm keep updating the estimative of the value function until it converges. After we have the optimium value function we can act optimally.

![alt text](imgs/value_iteration_orig.png "Game")

$$Q_\text{k+1}(s,a)=\sum_{\text{s'}}P(s'|s,a).(R(s,a,s') + \gamma V_k(s')) \text{     } \forall k \geq 0$$
$$V_k(s)=max_a(Q_k(s,a)) \text{     } \forall k > 0$$

$$V^*(s)=max_a[R(s,a)+\gamma . \sum_{\text{s'}}P(s'|s,a).V^*(s')]$$

You can either save the $V(s)$ array or the $Q(s,a)$ array. Saving the V array results in less storage, but it is more difficult to determine an optimal action, and one more iteration is needed to determine which action results in the greatest value. Storing the Q array you have imediatelly the action with biggest value to each state.

### References
* https://medium.com/@m.alzantot/deep-reinforcement-learning-demystified-episode-0-2198c05a6124
* https://medium.com/@m.alzantot/deep-reinforcement-learning-demysitifed-episode-2-policy-iteration-value-iteration-and-q-978f9e89ddaa
* https://artint.info/html/ArtInt_227.html
* https://pt.overleaf.com/learn/latex/Integrals,_sums_and_limits
* https://www.youtube.com/watch?v=c5X7Xj1iYak
* https://en.wikipedia.org/wiki/Bellman_equation
* https://www.youtube.com/watch?v=KovN7WKI9Y0
* https://www.youtube.com/watch?v=glHKJ359Cnc
* https://www.youtube.com/watch?v=cTu7mvRE354

### Import libraries and define Hyperparameters

In [1]:
import numpy as np
import time
import gym
from gym import wrappers
from IPython.display import clear_output

# Hyper parameters
gamma = 0.99

### Load OpenAI Gym Environment

In [2]:
#env_name  = 'FrozenLake8x8-v0'
#env_name = 'FrozenLake-v0'
env_name = 'Taxi-v2'

env = gym.make(env_name)

/Users/leonardo_a/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


### Access Environment Model
This part of the code will expose a dictionary $P(state,action)$ that will return a list of tupples $(prob,\text{next_state},reward,done)$, this will be the list of possible next states from that particular (state,action) pair, while P exposes the MDP of the environment.

For example this code would return the same thing
```python
state_reset = env.reset()
print('reset state:',state_reset)
action=0
next_state, reward, done, info = env.step(action)
print('############')
print('Next state:',next_state)
print('Reward:',reward)
print('done:',done)
print('info:',info)

#reset state: 0
############
#Next state: 0
#Reward: 0.0
#done: False
#info: {'prob': 0.3333333333333333}

# Return all the possible states with the tupple (probability, next_state, reward, done)
env.P[state_reset][action]
# [(0.3333333333333333, 0, 0.0, False),
# (0.3333333333333333, 0, 0.0, False),
# (0.3333333333333333, 8, 0.0, False)]
```

Just few environments expose their MDP:
* FrozenLake-v0
* FrozenLake8x8-v0
* Taxi-v2

In [3]:
# Have access to the MDP of the environment
env = env.unwrapped

### Display search space size

In [4]:
num_states = None
num_actions = None
try:
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    print('Search space:',num_states*num_actions)
except:
    num_states = np.prod(env.observation_space.shape)
    num_actions = np.prod(env.action_space.shape)
    print('Search space:',num_states*num_actions)

Search space: 3000


### Get a policy from optimum Value Function
As mentioned earlier if we have the optimum value function we can get the optimum policy. If you have the action-value function the job is actually easier but if you have the value function you need to do the following:
* Select a particular state
* for each action from that particular state
* Query from the transition function all the possible states from that particular state,action
* Choose the action with biggest value

 for each state s do 
    
    π[s] = argmaxa ∑s' P(s'|s,a) (R(s,a,s')+ γVk[s']) 
 
 return π,Vk

In [5]:
def extract_policy_from_value(v, gamma = 1.0):
    policy = np.zeros(env.observation_space.n)
    # For each state
    for s in range(env.observation_space.n):
        #for a in range(env.action_space.n):
            #policy[s] = np.argmax(sum([p*(r + gamma*v[s_]) for p,s_,r,_ in env.P[s][a]]))
        policy[s] = np.argmax([sum([p*(r + gamma*v[s_]) for p,s_,r,_ in env.P[s][a]]) for a in range(env.action_space.n)])
    return policy

### Value Iteration Algorithm

#### Bellman Optimality Equation
$$V^*(s)=max_a(Q^{\pi^*}(s,a))$$

The value iteration algorithm has the following structure:

1. Start with a zeroed value function
2. Use the bellman optimality equation to get an estimative of the optimum value function.
3. Repeat step 2 until the value function doesnt change anymore.
![alt text](imgs/value_iteration.png "Game")

In [6]:
def value_iteration(env, gamma = 1.0, max_iterations = 100000):
    # Initialize Value function with zeros
    v = np.zeros(env.observation_space.n)  # initialize value-function
    q = np.zeros((env.observation_space.n, env.action_space.n))
    
    # Big number of iterations
    for i in range(max_iterations):
        prev_v = np.copy(v)
        # For each state
        for s in range(env.observation_space.n):
            for a in range(env.action_space.n):
                q[s,a] = sum([p*(r + gamma*prev_v[s_]) for p, s_, r, _ in env.P[s][a]])
            # Convert action-value(Q) function into value function(V)
            v[s] = max(q[s,:])
        
        # Check convergence (Basically check if the value function didn't change much)
        if (np.sum(np.fabs(prev_v - v)) <= np.finfo(float).eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

### Run the Value Iteration Algorithm and get the best policy

In [7]:
%%time
# Run the value iteration
optimal_v = value_iteration(env, gamma);
policy = extract_policy_from_value(optimal_v, gamma)

Value-iteration converged at iteration# 3270.
CPU times: user 13.6 s, sys: 28.2 ms, total: 13.6 s
Wall time: 13.6 s


### Test learned policy

In [8]:
state = env.reset()
done = False
step = 0
while not done:
    action = int(policy[state])
    next_state, reward, done, info = env.step(action)
    state = next_state
    step += 1
    print("Step: {}".format(step))
    env.render()
    clear_output(wait=True)
    time.sleep(0.2)

Step: 16
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
